# PR checks

In [1]:
import pennylane as qml

## PR 1

https://github.com/PennyLaneAI/pennylane/pull/5348

- Checking docstrings:

In [2]:
dev = qml.device('default.qubit', wires=[0, 1, 2])

In [3]:
@qml.compile                ### DEVE ESSERE AGGIUNTO IL DECORATORE @qml.compile
@qml.qnode(device=dev)
def circuit(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(z, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.RX(y, wires=2)
    qml.Y(2)
    qml.CY(wires=[1, 2])
    return qml.expval(qml.Z(0))

In [4]:
def qfunc(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(z, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.RX(y, wires=2)
    qml.Y(2)
    qml.CY(wires=[1, 2])
    return qml.expval(qml.Z(0))

In [5]:
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)(0.2, 0.3, 0.4))

0: ──H──RX(0.40)────╭X──────────RX(0.20)─╭X────┤  <Z>
1: ──H───────────╭X─╰●───────────────────╰●─╭●─┤     
2: ──H──RZ(0.40)─╰●──RZ(-0.40)──RX(0.30)──Y─╰Y─┤     


In [6]:
compiled_qnode = qml.compile(qnode)
print(qml.draw(compiled_qnode)(0.2, 0.3, 0.4))

0: ──H──RX(0.60)─────────────────┤  <Z>
1: ──H─╭X──────────────────╭●────┤     
2: ──H─╰●─────────RX(0.30)─╰Y──Y─┤     


In [7]:
from functools import partial ### DEVE ESSERE AGGIUNTO

compiled_qnode = qml.compile(
    qnode,
    pipeline=[
        partial(qml.transforms.commute_controlled, direction="left"),
        partial(qml.transforms.merge_rotations, atol=1e-6),
        qml.transforms.cancel_inverses
    ],
    basis_set=["CNOT", "RX", "RY", "RZ"],
    num_passes=2
)

print(qml.draw(compiled_qnode)(0.2, 0.3, 0.4))

0: ──RZ(1.57)──RX(1.57)──RZ(1.57)──RX(0.60)─────────────────────────────────────────────────────
1: ──RZ(1.57)──RX(1.57)──RZ(1.57)─╭X─────────RZ(1.57)─────────────────────────────────────────╭●
2: ──RZ(1.57)──RX(1.57)──RZ(1.57)─╰●─────────RX(0.30)──RZ(1.57)──RY(3.14)──RZ(1.57)──RY(1.57)─╰X

────────────────┤  <Z>
─────────────╭●─┤     
───RY(-1.57)─╰X─┤     


Due cambiamenti da apportare. Da notare che quel coglione ha linkato la doc stable, ma bisogna guardare la 'latest'

## PR 2

https://github.com/PennyLaneAI/pennylane/pull/5339

Questa is una rimozione from deprecation and removal, quindi penso sia tutto a posto

## PR 3

https://github.com/PennyLaneAI/pennylane/pull/5347

In [8]:
op = qml.evolve(qml.X(0), coeff=2)
op

Evolution(-2j PauliX)

In [9]:
coeffs = [lambda p, t: p * t for _ in range(4)]
ops = [qml.X(i) for i in range(4)]

# ParametrizedHamiltonian
H = qml.dot(coeffs, ops)

# ParametrizedEvolution
ev = qml.evolve(H)

In [10]:
ev

ParametrizedEvolution(wires=[0, 1, 2, 3])

In [11]:
qml.matrix(ev([1., 2., 3., 4.], t=[0, 4]))

Array([[ 0.04929843+0.j        ,  0.        -0.03258221j,
         0.        +0.10526j   ,  0.06956796+0.j        ,
         0.        -0.01481935j, -0.00979378+0.j        ,
         0.03164092+0.j        ,  0.        -0.02091208j,
         0.        +0.33525273j,  0.22157252+0.j        ,
        -0.7158268 +0.j        ,  0.        +0.4730951j ,
         0.10077609+0.j        ,  0.        -0.06660434j,
         0.        +0.21517362j,  0.14221084+0.j        ],
       [ 0.        -0.03258221j,  0.04929843+0.j        ,
         0.06956796+0.j        ,  0.        +0.10526j   ,
        -0.00979378+0.j        ,  0.        -0.01481935j,
         0.        -0.02091208j,  0.03164092+0.j        ,
         0.22157252+0.j        ,  0.        +0.33525273j,
         0.        +0.4730951j , -0.7158268 +0.j        ,
         0.        -0.06660434j,  0.10077609+0.j        ,
         0.14221084+0.j        ,  0.        +0.21517362j],
       [ 0.        +0.10526j   ,  0.06956796+0.j        ,
         0.0

In [12]:
import jax

dev = qml.device("default.qubit.jax", wires=4)
@jax.jit
@qml.qnode(dev, interface="jax")
def circuit(params):
    qml.evolve(H)(params, t=[0, 10])
    return qml.expval(qml.Z(0))

In [13]:
params = [1., 2., 3., 4.]
circuit(params)

Array(0.8609962, dtype=float32)

In [14]:
jax.grad(circuit)(params)

[Array(50.85762, dtype=float32),
 Array(-5.347459e-05, dtype=float32),
 Array(-9.281231e-05, dtype=float32),
 Array(-0.00013054, dtype=float32)]

## PR 4

https://github.com/PennyLaneAI/pennylane/pull/5430/files

## PR 5

https://github.com/PennyLaneAI/pennylane/pull/5422

In [16]:
import jax

jax.config.update("jax_enable_x64", True)

dev = qml.device("default.qubit", wires=2, shots=100)

@qml.qnode(dev, interface="jax")
def circuit(x):
    qml.RX(x, wires=0)
    return qml.sample(qml.PauliX(0) @ qml.PauliY(1))

results = jax.jit(circuit)(jax.numpy.array(0.123, dtype=jax.numpy.float64))

import jax

jax.config.update("jax_enable_x64", True)

dev = qml.device("default.qubit", wires=2, shots=100)

@qml.qnode(dev, interface="jax")
def circuit(x):
    qml.RX(x, wires=0)
    return qml.sample(qml.PauliX(0) @ qml.PauliY(1))

results = jax.jit(circuit)(jax.numpy.array(0.123, dtype=jax.numpy.float64))

## PR 6

https://github.com/PennyLaneAI/pennylane/pull/5459

## PR 7

https://github.com/PennyLaneAI/pennylane/pull/5441

In [17]:
dev = qml.device("default.qubit")

@qml.qnode(dev)
def circuit(x, y):
    qml.RX(x, wires=0)
    qml.RY(y, wires=1)
    m0 = qml.measure(1)
    return (
        qml.sample(m0), qml.expval(m0), qml.var(m0), qml.probs(op=m0), qml.counts(op=m0),
    )

In [18]:
circuit(1.0, 2.0, shots=1000)

(array([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 

## PR 8

https://github.com/PennyLaneAI/pennylane/pull/5455

Modifica da fare al changelog

## PR 9

https://github.com/PennyLaneAI/pennylane/pull/

In [19]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit")
@qml.qnode(dev)
def cost(params):
    qml.RX(params[0], wires=0)
    qml.CRY(params[1], wires=[0, 1])
    return qml.expval(qml.Z(0) @ qml.Z(1))

params = np.random.rand(2)
opt = qml.QNSPSAOptimizer(stepsize=5e-2)

for _ in range(3):
    print(type(params))
    params, loss = opt.step_and_cost(cost, params)
    #params = np.array(params) # this fixes the error

<class 'pennylane.numpy.tensor.tensor'>
<class 'pennylane.numpy.tensor.tensor'>
<class 'pennylane.numpy.tensor.tensor'>


## PR 10

https://github.com/PennyLaneAI/pennylane/pull/5450